In [1]:
import os
import shutil

if not os.path.isdir("./libs"):
    os.mkdir("./libs")

In [2]:
%%writefile ./libs/util_bd.py
import mysql.connector
import myloginpath
import pandas as pd

def resultados_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    resultado = cursor.fetchall()
    conn.close()
    return resultado

def ejecutar_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    conn.close()

def mostrar_resultado_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    df = pd.read_sql_query(query, conn)
    display(df)
    conn.close()

Writing ./libs/util_bd.py


In [3]:
%%writefile ./libs/util_fasta.py
import os

def generar_fasta(secuencias, archivo, tamanio_por_linea=80):
    t_tamanio = tamanio_por_linea
    f = open(archivo ,"w+")
    for transcrito in secuencias:
        f.write(">%s\n" % (transcrito[0].strip().upper()))
        seq = transcrito[1]
        t_partes = [seq[i:i+t_tamanio] for i in range(0, len(seq), t_tamanio)]
        for t_parte in t_partes:
            f.write("%s\n" % (t_parte))
    f.close()

def leer_fasta(archivo, limite = 0):
    transcritos = {}
    cod_secuencia = ""
    secuencia = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if secuencia != "":
                transcritos[cod_secuencia] = secuencia
                secuencia = ""
                limite -= 1
                if limite == 0:
                    break
            cod_secuencia = linea.rstrip("\n").lstrip(">").strip().upper()
        else:
            secuencia += linea.rstrip("\n")
    if secuencia != "":
        transcritos[cod_secuencia] = secuencia
        secuencia = ""
    f.close()
    return transcritos

def leer_fasta_list(archivo, limite = 0):
    transcritos = list()
    cod_secuencia = ""
    secuencia = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if secuencia != "":
                transcritos.append((cod_secuencia, secuencia))
                secuencia = ""
                limite -= 1
                if limite == 0:
                    break
            cod_secuencia = linea.rstrip("\n").lstrip(">").strip().upper()
        else:
            secuencia += linea.rstrip("\n")
    if secuencia != "":
        transcritos.append((cod_secuencia, secuencia))
        secuencia = ""
    f.close()
    return transcritos

Writing ./libs/util_fasta.py


In [4]:
%%writefile ./libs/util_caracteristicas.py
import os
import util_fasta
from Bio.SeqUtils import GC
import csv
from sklearn.externals.joblib import dump, load

def generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat):
    _generar_modelo_CPAT_hexamer(archivo_lncRNA, archivo_CDS, carpeta_cpat)
    _generar_modelo_CPAT_logit(archivo_lncRNA, archivo_PCT, carpeta_cpat)

def _generar_modelo_CPAT_hexamer(archivo_lncRNA, archivo_CDS, carpeta_cpat):
    script = "~/anaconda3/bin/make_hexamer_tab.py"
    fasta_cds = "'" + archivo_CDS + "'" 
    fasta_lncRNA = "'" + archivo_lncRNA + "'"
    salida = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    comando = "{} -c {} -n {} > {}".format(script, fasta_cds, fasta_lncRNA, salida)
    os.system(comando)
    
def _generar_modelo_CPAT_logit(archivo_lncRNA, archivo_PCT, carpeta_cpat):
    script = "~/anaconda3/bin/make_logitModel.py"
    hexamer = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    fasta_pct = "'" + archivo_PCT + "'" 
    fasta_lncRNA = "'" + archivo_lncRNA + "'"
    salida = "'" + carpeta_cpat + "/fold" + "'"
    comando = "{} -x {} -c {} -n {} -o {}".format(script, hexamer, fasta_pct, fasta_lncRNA, salida)
    os.system(comando)
    
def ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida):
    script = "~/anaconda3/bin/diamond"
    diamond_bd = "'" + diamond_db + "'"
    salida = "'" + archivo_salida + "'"
    comando = "{} blastx -d {} -q {} -o {} -k 5 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore".format(script, diamond_bd, archivo_entrada, salida)
    os.system(comando)

def ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida):
    script = "~/anaconda3/bin/cpat.py"
    logit = "'" + carpeta_cpat + "/fold.logit.RData" + "'"
    hexamer = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    salida = "'" + archivo_salida + "'"
    comando = "{} -g {} -d {} -x {} -o {}".format(script, archivo_entrada, logit, hexamer, salida)
    os.system(comando)
    
def generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida):
    transcritos = util_fasta.leer_fasta(archivo_entrada)
    transcript_dict = {}
    for k in transcritos.keys():
        transcript_dict[k.strip().upper()] = {
            "length" : len(transcritos[k]),
            "gc" : GC(transcritos[k]),
            "orf_length" : 0,
            "orf_coverage" : float(0),
            "hexamer_score" : float(0),
            "fickett_score" : float(0),
            "identity" : float(0),
            "align_length" : float(0),
            "align_perc_len" : float(0),
            "align_perc_orf" : float(0)
        }
    
    #adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    with open(archivo_cpat, "r") as f:
        cpat_reader = csv.reader(f, delimiter=("\t"))
        next(cpat_reader, None) # skip header
        for row in cpat_reader:
            cod_secuencia = row[0]
            transcript_dict[cod_secuencia]["orf_length"] = float(row[2])
            transcript_dict[cod_secuencia]["orf_coverage"] = float(row[2])/float(transcript_dict[cod_secuencia]["length"])
            transcript_dict[cod_secuencia]["fickett_score"] = float(row[3])
            transcript_dict[cod_secuencia]["hexamer_score"] = float(row[4])
    
    with open(archivo_diamond, "r") as f:
        tab_reader = csv.reader(f, delimiter=("\t"))
        line_1 = next(tab_reader)
        first = line_1[0].upper()
        score = [float(line_1[9])]
        with_len = [[first, float(line_1[1]), float(line_1[2]), float(line_1[3]), float(line_1[9])]] # name identity length frame score
        for row in tab_reader:
            if row[0].upper() == first:
                score.append(float(row[9]))
                with_len.append([row[0].upper(), float(row[1]), float(row[2]), float(row[3]), float(row[9])])
            else:
                transcript_dict[first]["identity"] = float(0)
                transcript_dict[first]["align_length"] = float(0)
                max_value = max(score)
                max_index = score.index(max_value)
                max_len_ident = with_len[max_index]
                if max_len_ident[3] > 0:
                    transcript_dict[first]["identity"] = float(max_len_ident[1])
                    transcript_dict[first]["align_length"] = float(max_len_ident[2])
                    transcript_dict[first]["align_perc_len"] = float(transcript_dict[first]["align_length"]/transcript_dict[first]["length"])
                    transcript_dict[first]["align_perc_orf"] = (0 if transcript_dict[first]["orf_length"] == 0 else float(transcript_dict[first]["align_length"]/transcript_dict[first]["orf_length"]))
                score = [float(row[9])]
                first = row[0].upper()
                with_len = [[first, float(row[1]), float(row[2]), float(row[3]), float(row[9])]]
        transcript_dict[first]["identity"] = float(0)
        transcript_dict[first]["align_length"] = float(0)
        max_value = max(score)
        max_index = score.index(max_value)
        max_len_ident = with_len[max_index]
        if max_len_ident[3] > 0:
            transcript_dict[first]["identity"] = float(max_len_ident[1])
            transcript_dict[first]["align_length"] = float(max_len_ident[2])
    #fin de código adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    
    dump(transcript_dict, archivo_salida)

def generar_features(archivo_entrada, features_base, archivo_cpat, archivo_salida):
    transcritos = util_fasta.leer_fasta(archivo_entrada)
    features_globales = load(features_base)
    transcript_dict = {}
    for k in transcritos.keys():
        transcript_dict[k.strip().upper()] = {
            "length" : features_globales[k.strip().upper()]["length"],
            "gc" : features_globales[k.strip().upper()]["gc"],
            "orf_length" : features_globales[k.strip().upper()]["orf_length"],
            "orf_coverage" : features_globales[k.strip().upper()]["orf_coverage"],
            "hexamer_score" : float(0),
            "fickett_score" : float(0),
            "identity" : features_globales[k.strip().upper()]["identity"],
            "align_length" : features_globales[k.strip().upper()]["align_length"],
            "align_perc_len" : features_globales[k.strip().upper()]["align_perc_len"],
            "align_perc_orf" : features_globales[k.strip().upper()]["align_perc_orf"]
        }
    
    with open(archivo_cpat, "r") as f:
        cpat_reader = csv.reader(f, delimiter=("\t"))
        next(cpat_reader, None) # skip header
        for row in cpat_reader:
            cod_secuencia = row[0].strip().upper()
            if cod_secuencia in transcript_dict:
                transcript_dict[cod_secuencia]["fickett_score"] = float(row[3])
                transcript_dict[cod_secuencia]["hexamer_score"] = float(row[4])
                
    dump(transcript_dict, archivo_salida)

Writing ./libs/util_caracteristicas.py


In [1]:
%%writefile ./libs/util_modelo_final.py
import os
import shutil
import util_fasta, util_caracteristicas
from sklearn.externals.joblib import Parallel, delayed, dump, load
import hashlib
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, precision_recall_curve, average_precision_score, roc_curve, roc_auc_score
import hashlib
import util_fasta
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

class Tesis2():
    def __init__(self, carpeta_base=".", n_jobs=-1, verbose=0, tuned_parameters=[{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3], 'svc__C': [0.1,0.5,0.9,2]}], score = ['accuracy','precision','recall']):
        self.carpeta_base = carpeta_base
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.tuned_parameters = tuned_parameters
        self.score = score
        self.carpeta_data_base = self.carpeta_base + "/data"
        self.carpeta_fold_base = self.carpeta_base + "/folds"
        self.carpeta_modelo_base = self.carpeta_base + "/modelo_final"
        if not os.path.isdir(self.carpeta_base):
            os.mkdir(self.carpeta_base)
        self.diamond_db = "./feature_engine/Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
        self.modelo_final_generado = False
        self.modelo_referencial_generado = False
            
    def generar_modelo_final(self):
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("*************** Generando llaves ****************")
        if (self.verbose > 1): print("*************************************************")
        self.generar_llaves_clases()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("***************** Armando folds *****************")
        if (self.verbose > 1): print("*************************************************")
        self.armar_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("***** Generando modelo cpat para cada fold ******")
        if (self.verbose > 1): print("*************************************************")
        self.generar_cpats_de_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("*** Ejecutando cpat y diamond sobre los folds ***")
        if (self.verbose > 1): print("*************************************************")
        self.ejecutar_cpat_diamond_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("************* Serializando features *************")
        if (self.verbose > 1): print("*************************************************")
        self.generar_features_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("************ Generando modelo final *************")
        if (self.verbose > 1): print("*************************************************")
        self.entrenar_modelo_final()
        self.modelo_final_generado = True
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("******** Limpiando archivos intermedios *********")
        if (self.verbose > 1): print("*************************************************")
        self.limpiar_archivos_intermedios()
        if (self.verbose > 1):
            print("*************************************************")
            print("************* Mostrando resultados **************")
            print("*************************************************")
            self.mostrar_resultados()
            
    def generar_modelo_final_keras(self):
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("*************** Generando llaves ****************")
        if (self.verbose > 1): print("*************************************************")
        self.generar_llaves_clases()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("***************** Armando folds *****************")
        if (self.verbose > 1): print("*************************************************")
        self.armar_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("***** Generando modelo cpat para cada fold ******")
        if (self.verbose > 1): print("*************************************************")
        self.generar_cpats_de_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("*** Ejecutando cpat y diamond sobre los folds ***")
        if (self.verbose > 1): print("*************************************************")
        self.ejecutar_cpat_diamond_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("************* Serializando features *************")
        if (self.verbose > 1): print("*************************************************")
        self.generar_features_folds()
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("************ Generando modelo final *************")
        if (self.verbose > 1): print("*************************************************")
        self.entrenar_modelo_final_keras()
        self.modelo_final_generado = True
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("******** Limpiando archivos intermedios *********")
        if (self.verbose > 1): print("*************************************************")
        self.limpiar_archivos_intermedios()
        if (self.verbose > 1):
            print("*************************************************")
            print("************* Mostrando resultados **************")
            print("*************************************************")
            self.mostrar_resultados()
            
    def generar_modelos_referenciales(self):
        if (self.verbose > 1): print("*************************************************")
        if (self.verbose > 1): print("******* Generando modelos referenciales *********")
        if (self.verbose > 1): print("*************************************************")
        self.entrenar_modelos_referenciales()
        self.modelo_referencial_generado = True
        if (self.modelo_final_generado and self.verbose > 1):
            print("*************************************************")
            print("************* Mostrando resultados **************")
            print("*************************************************")
            self.mostrar_resultados_referencial_vs_final()
        
    def carpeta_data(self):
        return self.carpeta_data_base

    def carpeta_fold(self):
        return self.carpeta_fold_base

    def carpeta_modelo(self):
        return self.carpeta_modelo_base
    
    def folder_clase(self, num_clase):
        return self.carpeta_data() + "/clase_" + str(num_clase)

    def archivo_clase(self, num_clase, tipo):
        return self.folder_clase(num_clase) + "/" + tipo + ".fa"

    def obtener_num_clases(self):
        num_clases = 0
        if not os.path.isfile(self.carpeta_base + "/num_clases.bin"):
            while os.path.isdir(self.folder_clase(num_clases + 1)):
                num_clases = num_clases + 1
            dump(num_clases, self.carpeta_base + "/num_clases.bin")
        num_clases = load(self.carpeta_base + "/num_clases.bin")
        return num_clases

    def iterador_clases(self):
        return range(1, self.obtener_num_clases() + 1)

    def obtener_primera_secuencia(self, num_clase):
        secuencias = util_fasta.leer_fasta(self.archivo_clase(num_clase, "lncRNA"), 1)
        return list(secuencias.keys())[0]

    def obtener_todas_las_secuencias(self):
        return {num_clase : self.obtener_primera_secuencia(num_clase) for num_clase in self.iterador_clases()}

    def generar_llaves_clases(self):
        secuencias = self.obtener_todas_las_secuencias()
        llaves = {num_clase : "" for num_clase in secuencias.keys()}
        llaves[0] = "" #llave cero corresponde a todo el universo
        for i_clase in self.iterador_clases():
            llaves[0] += secuencias[i_clase]
            for j_clase in self.iterador_clases():
                if (i_clase != j_clase):
                    llaves[j_clase] += secuencias[i_clase]
        llaves[0] = hashlib.sha224(llaves[0].encode()).hexdigest()
        for i_clase in self.iterador_clases():
            llaves[i_clase] = hashlib.sha224(llaves[i_clase].encode()).hexdigest()
        dump(llaves, self.carpeta_base + "/llaves_clases.bin")

    def obtener_llaves_clases(self):
        return load(self.carpeta_base + "/llaves_clases.bin")

    def carpeta_fold_clase(self, llave):
        return self.carpeta_fold() + "/fold_clase_" + str(llave)

    def archivo_fold_clase(self, llave, tipoTrainTest, tipoRNA):
        return self.carpeta_fold_clase(llave) + "/" + tipoTrainTest + "/" + tipoRNA + ".fa"

    def armar_fold_final(self, tipo):
        llave = self.obtener_llaves_clases()[0]
        with open(self.archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
            for num_clase in self.iterador_clases():
                with open(self.archivo_clase(num_clase, tipo), "r") as infile:
                    for inline in infile:
                        outfile.write(inline)

    def armar_fold_clase(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        os.mkdir(self.carpeta_fold_clase(llave))
        os.mkdir(self.carpeta_fold_clase(llave) + "/train")
        for tipo in ["lncRNA", "PCT", "CDS"]:
            with open(self.archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
                for j_num_clase in self.iterador_clases():
                    if num_clase != j_num_clase:
                        with open(self.archivo_clase(j_num_clase, tipo)) as infile:
                            for inline in infile:
                                outfile.write(inline)
        os.mkdir(self.carpeta_fold_clase(llave) + "/test")
        for tipo in ["lncRNA", "PCT"]:
            with open(self.archivo_fold_clase(llave, "test", tipo), "w+") as outfile:
                with open(self.archivo_clase(num_clase, tipo)) as infile:
                    for inline in infile:
                        outfile.write(inline)
                    
    def armar_folds(self):
        if os.path.isdir(self.carpeta_fold()):
            shutil.rmtree(self.carpeta_fold())
        os.mkdir(self.carpeta_fold())
        llave = self.obtener_llaves_clases()[0]
        if not os.path.isdir(self.carpeta_fold_clase(llave)):
            os.mkdir(self.carpeta_fold_clase(llave))
        if not os.path.isdir(self.carpeta_fold_clase(llave) + "/train"):
            os.mkdir(self.carpeta_fold_clase(llave) + "/train")

        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_armar_fold_final)(self, tipo) for tipo in ["lncRNA", "PCT", "CDS"])
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_armar_fold_clase)(self, num_clase) for num_clase in self.iterador_clases())
        
    def carpeta_fold_cpat(self, llave):
        return self.carpeta_fold_clase(llave) + "/cpat"

    def generar_cpat_fold(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        archivo_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA")
        archivo_PCT = self.archivo_fold_clase(llave, "train", "PCT")
        archivo_CDS = self.archivo_fold_clase(llave, "train", "CDS")
        carpeta_cpat = self.carpeta_fold_cpat(llave)
        if not os.path.isdir(carpeta_cpat):
            os.mkdir(carpeta_cpat)
        util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

    def generar_cpat_final(self):
        llave = self.obtener_llaves_clases()[0]
        archivo_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA")
        archivo_PCT = self.archivo_fold_clase(llave, "train", "PCT")
        archivo_CDS = self.archivo_fold_clase(llave, "train", "CDS")
        carpeta_cpat = self.carpeta_fold_cpat(llave)
        if not os.path.isdir(carpeta_cpat):
            os.mkdir(carpeta_cpat)
        util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

    def limpieza_archivos_CDS(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        os.remove(self.archivo_fold_clase(llave, "train", "CDS"))
        
    def generar_cpats_de_folds(self):
        self.generar_cpat_final()
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_generar_cpat_fold)(self, num_clase) for num_clase in self.iterador_clases())
        self.limpieza_archivos_CDS(0)
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_limpieza_archivos_CDS)(self, num_clase) for num_clase in self.iterador_clases())
    
    def ejecutar_cpat_fold(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        archivo_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA")
        archivo_PCT = self.archivo_fold_clase(llave, "train", "PCT")
        carpeta_cpat = self.carpeta_fold_cpat(llave)
        util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
        os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
        os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
        util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
        os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
        os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")

    def ejecutar_cpat_diamond_final(self):
        llave = self.obtener_llaves_clases()[0]
        archivo_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA")
        archivo_PCT = self.archivo_fold_clase(llave, "train", "PCT")
        diamond_db = self.diamond_db
        carpeta_cpat = self.carpeta_fold_cpat(llave)
        util_caracteristicas.ejecutar_diamond(archivo_lncRNA, diamond_db, archivo_lncRNA.replace(".fa", ".dmnd"))
        util_caracteristicas.ejecutar_diamond(archivo_PCT, diamond_db, archivo_PCT.replace(".fa", ".dmnd"))
        util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
        os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
        os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
        util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
        os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
        os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")
        
    def ejecutar_cpat_diamond_folds(self):
        self.ejecutar_cpat_diamond_final()
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_ejecutar_cpat_fold)(self, num_clase) for num_clase in self.iterador_clases())
        
    def archivo_features_clase(self, llave, tipoTrainTest, tipoRNA):
        return self.archivo_fold_clase(llave, tipoTrainTest, tipoRNA).replace(".fa", ".ft")

    def generar_features_fold(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        features_base_lncRNA = self.archivo_features_clase(self.obtener_llaves_clases()[0], "train", "lncRNA")
        features_base_PCT = self.archivo_features_clase(self.obtener_llaves_clases()[0], "train", "PCT")
        for tipo in ["train", "test"]:
            archivo_lncRNA = self.archivo_fold_clase(llave, tipo, "lncRNA")
            archivo_PCT = self.archivo_fold_clase(llave, tipo, "PCT")
            archivo_cpat_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA").replace(".fa", ".cpat")
            archivo_cpat_PCT = self.archivo_fold_clase(llave, "train", "PCT").replace(".fa", ".cpat")
            salida_lncRNA = self.archivo_features_clase(llave, tipo, "lncRNA")
            salida_PCT = self.archivo_features_clase(llave, tipo, "PCT")
            util_caracteristicas.generar_features(archivo_lncRNA, features_base_lncRNA, archivo_cpat_lncRNA, salida_lncRNA)
            util_caracteristicas.generar_features(archivo_PCT, features_base_PCT, archivo_cpat_PCT, salida_PCT)

    def generar_features_final(self):
        llave = self.obtener_llaves_clases()[0]
        archivo_lncRNA = self.archivo_fold_clase(llave, "train", "lncRNA")
        archivo_PCT = self.archivo_fold_clase(llave, "train", "PCT")
        salida_lncRNA = self.archivo_features_clase(llave, "train", "lncRNA")
        salida_PCT = self.archivo_features_clase(llave, "train", "PCT")
        util_caracteristicas.generar_features_base(archivo_lncRNA, archivo_lncRNA.replace(".fa", ".cpat"), archivo_lncRNA.replace(".fa", ".dmnd"), salida_lncRNA)
        util_caracteristicas.generar_features_base(archivo_PCT, archivo_PCT.replace(".fa", ".cpat"), archivo_PCT.replace(".fa", ".dmnd"), salida_PCT)
        
    def generar_features_folds(self):
        self.generar_features_final()
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_generar_features_fold)(self, num_clase) for num_clase in self.iterador_clases())
        
    def obtener_data_entrenamiento(self):
        llave = self.obtener_llaves_clases()[0]
        codigos_lncRNA = util_fasta.leer_fasta_list(self.archivo_fold_clase(llave, "train", "lncRNA"))
        codigos_PCT = util_fasta.leer_fasta_list(self.archivo_fold_clase(llave, "train", "PCT"))

        codigos_lncRNA = [(x[0],"") for x in codigos_lncRNA]
        codigos_PCT = [(x[0],"") for x in codigos_PCT]
        cantidad_transcritos = len(codigos_lncRNA)//self.obtener_num_clases()

        y = ([1] * len(codigos_lncRNA)) + ([0] * len(codigos_PCT))
        groups = list()
        for _ in range(2):
            for num_clase in self.iterador_clases():
                groups += ([num_clase] * (cantidad_transcritos))
        return codigos_lncRNA + codigos_PCT, y, groups, cantidad_transcritos
        
    def obtener_data_entrenamiento_keras(self):
        llave = self.obtener_llaves_clases()[0]
        codigos_lncRNA = util_fasta.leer_fasta_list(self.archivo_fold_clase(llave, "train", "lncRNA"))
        codigos_PCT = util_fasta.leer_fasta_list(self.archivo_fold_clase(llave, "train", "PCT"))

        codigos_lncRNA = [(x[0],"") for x in codigos_lncRNA]
        codigos_PCT = [(x[0],"") for x in codigos_PCT]
        cantidad_transcritos = len(codigos_lncRNA)//self.obtener_num_clases()

        y = ([1] * len(codigos_lncRNA)) + ([0] * len(codigos_PCT))
        groups = list()
        for _ in range(2):
            for num_clase in self.iterador_clases():
                groups += ([num_clase] * (cantidad_transcritos))
        x = codigos_lncRNA + codigos_PCT
        
        reserva_x = list()
        reserva_y = list()
        reserva_groups = list()
        iremove = list()
        num_transcritos = len(x)
        num_transcritos_por_grupo = cantidad_transcritos
        for i in range(num_transcritos//(num_transcritos_por_grupo*2)):
            reserva_x.append(x[i * num_transcritos_por_grupo])
            reserva_y.append(y[i * num_transcritos_por_grupo])
            reserva_groups.append(groups[i * num_transcritos_por_grupo])
            iremove.insert(0, i * num_transcritos_por_grupo)
        for i in iremove:
            del x[i]
            del y[i]
            del groups[i]
        x, y, groups = shuffle(x, y, groups, random_state=7)
        for i in range(num_transcritos//(num_transcritos_por_grupo*2)):
            x.insert(i, reserva_x[i])
            y.insert(i, reserva_y[i])
            groups.insert(i, reserva_groups[i])
            
        return x, y, groups, cantidad_transcritos

    def entrenar_modelo_final(self):
        if os.path.isdir(self.carpeta_modelo()):
            shutil.rmtree(self.carpeta_modelo())
        os.mkdir(self.carpeta_modelo())
        X_train, y_train, groups, cantidad_transcritos = self.obtener_data_entrenamiento()
        svm_pipeline = Pipeline(steps=[('features', GeneradorFeatures(self, cantidad_transcritos, self.obtener_num_clases())), ('scaler', RobustScaler()), ('svc', SVC())])
        logo = LeaveOneGroupOut()
        clf = GridSearchCV(svm_pipeline, self.tuned_parameters, cv=logo, scoring=self.score, n_jobs=self.n_jobs, refit="accuracy", return_train_score = True, verbose=self.verbose)
        clf.fit(X_train, y_train, groups) #requerido por LeaveOneGroupOut
        resultado = {
            "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
            "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
            "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
        }
        dump(resultado, self.carpeta_modelo() + "/resultado.bin")
        dump(clf.best_params_, self.carpeta_modelo() + "/params.bin")
        dump(clf.cv_results_, self.carpeta_modelo() + "/cv_results.bin")
        dump(clf.best_estimator_, self.carpeta_modelo() + "/modelo.plk")

    def entrenar_modelo_final_keras(self):
        if os.path.isdir(self.carpeta_modelo()):
            shutil.rmtree(self.carpeta_modelo())
        os.mkdir(self.carpeta_modelo())
        X_train, y_train, groups, cantidad_transcritos = self.obtener_data_entrenamiento_keras()
        #X_train, y_train, groups = shuffle(X_train, y_train, groups, random_state=7)
        keras_pipeline = Pipeline(steps=[('features', GeneradorFeaturesKeras(self, cantidad_transcritos, self.obtener_num_clases())), ('scaler', RobustScaler()), ('keras', KerasClassifier(build_fn=crear_modelo_keras, verbose=0))])
        logo = LeaveOneGroupOut()
        clf = GridSearchCV(keras_pipeline, self.tuned_parameters, cv=logo, scoring=self.score, n_jobs=self.n_jobs, refit="accuracy", return_train_score = True, verbose=self.verbose)
        clf.fit(X_train, y_train, groups) #requerido por LeaveOneGroupOut
        resultado = {
            "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
            "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
            "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
        }
        dump(resultado, self.carpeta_modelo() + "/resultado.bin")
        dump(clf.best_params_, self.carpeta_modelo() + "/params.bin")
        dump(clf.cv_results_, self.carpeta_modelo() + "/cv_results.bin")
        dump(clf.best_estimator_, self.carpeta_modelo() + "/modelo.plk")
        
    def limpieza_archivos_finales_fasta_ruta(self, llave):
        shutil.rmtree(self.carpeta_fold_clase(llave))

    def limpieza_archivos_finales_fasta(self, num_clase):
        llave = self.obtener_llaves_clases()[num_clase]
        if num_clase == 0:
            shutil.rmtree(self.carpeta_fold_clase(llave) + "/train")
        else:
            self.limpieza_archivos_finales_fasta_ruta(llave)
    
    def limpiar_archivos_intermedios(self):
        self.limpieza_archivos_finales_fasta(0)
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_limpieza_archivos_finales_fasta)(self, num_clase) for num_clase in self.iterador_clases())
        self.features_pre_generados = False
        
    def mostrar_resultados(self):
        if (not self.modelo_final_generado):
            print("Debe generar el modelo final")
            return
        display(load(self.carpeta_modelo() + "/cv_results.bin"))
        display(load(self.carpeta_modelo() + "/params.bin"))
        display(load(self.carpeta_modelo() + "/resultado.bin"))
        if (self.modelo_referencial_generado):
            self.mostrar_resultados_referencial_vs_final()
        
    def devolver_resultado(self):
        return load(self.carpeta_modelo() + "/resultado.bin")
        
    def devolver_mejor_parametro(self):
        return load(self.carpeta_modelo() + "/params.bin")
        
    def devolver_cv_results(self):
        return load(self.carpeta_modelo() + "/cv_results.bin")
        
    def devolver_mejor_modelo(self):
        return load(self.carpeta_modelo() + "/modelo.plk")
        
    def preparar_data_modelo_referencial(self, num_clase):
        carpeta_base_referencial = self.carpeta_base + "/modelos_referenciales/clase_" + str(num_clase)
        if not os.path.isdir(carpeta_base_referencial):
            os.mkdir(carpeta_base_referencial)
        if not os.path.isdir(carpeta_base_referencial + "/data"):
            os.mkdir(carpeta_base_referencial + "/data")
        clase_positiva = util_fasta.leer_fasta_list(self.archivo_clase(num_clase, "lncRNA"))
        PCT = util_fasta.leer_fasta(self.archivo_clase(num_clase, "PCT"))
        CDS = util_fasta.leer_fasta(self.archivo_clase(num_clase, "CDS"))
        clase_negativa = list()
        for k in PCT.keys():
            clase_negativa.append((k, PCT[k], CDS[k]))
        X = clase_positiva + clase_negativa
        y = ([1] * len(clase_positiva)) + ([0] * len(clase_negativa))
        skf = StratifiedKFold(n_splits=10)
        isplit = 1
        for _, test in skf.split(X, y):
            split_lncRNA = list()
            split_PCT = list()
            split_CDS = list()
            for itest in test:
                if y[itest] == 1:
                    split_lncRNA.append(X[itest])
                else:
                    split_PCT.append((X[itest][0], X[itest][1]))
                    split_CDS.append((X[itest][0], X[itest][2]))
            if not os.path.isdir(carpeta_base_referencial + "/data/clase_" + str(isplit)):
                os.mkdir(carpeta_base_referencial + "/data/clase_" + str(isplit))
            util_fasta.generar_fasta(split_lncRNA, carpeta_base_referencial + "/data/clase_" + str(isplit) + "/lncRNA.fa")
            util_fasta.generar_fasta(split_PCT, carpeta_base_referencial + "/data/clase_" + str(isplit) + "/PCT.fa")
            util_fasta.generar_fasta(split_CDS, carpeta_base_referencial + "/data/clase_" + str(isplit) + "/CDS.fa")
            isplit += 1
            
    def instanciar_modelo_referencial(self, num_clase):
        carpeta_base_referencial = self.carpeta_base + "/modelos_referenciales/clase_" + str(num_clase)
        return Tesis2(carpeta_base=carpeta_base_referencial, n_jobs=self.n_jobs, verbose=0, tuned_parameters=self.tuned_parameters, score=self.score)
        
    def crear_modelo_referencial(self, num_clase):
        carpeta_base_referencial = self.carpeta_base + "/modelos_referenciales/clase_" + str(num_clase)
        self.preparar_data_modelo_referencial(num_clase)
        tesis2 = self.instanciar_modelo_referencial(num_clase)
        tesis2.generar_modelo_final()
        shutil.rmtree(carpeta_base_referencial + "/data")
    
    def entrenar_modelos_referenciales(self):
        if not os.path.isdir(self.carpeta_base + "/modelos_referenciales"):
            os.mkdir(self.carpeta_base + "/modelos_referenciales")
        Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(delayed(wrapper_crear_modelo_referencial)(self, num_clase) for num_clase in self.iterador_clases())
        
    def obtener_resultados_referencial_vs_final(self):
        resultado_referencial = list()
        resultado_final = list()
        if (not self.modelo_final_generado) or (not self.modelo_referencial_generado):
            print("Debe generar ambos modelos para obtener resultados comparativos")
            return resultado_referencial, resultado_final 
        parametros = self.devolver_mejor_parametro()
        resultados = self.devolver_cv_results()
        i_seleccionado = 0
        i_iter = 0
        for param in resultados["params"]:
            if parametros == param:
                i_seleccionado = i_iter
            i_iter += 1

        for i in range(self.obtener_num_clases()):
            resultado_referencial.append(self.instanciar_modelo_referencial(i+1).devolver_resultado()["accuracy"])
            resultado_final.append(resultados["split" + str(i) + "_test_accuracy"][i_seleccionado])
        return resultado_referencial, resultado_final
        
    def mostrar_resultados_referencial_vs_final(self):
        if (not self.modelo_final_generado) or (not self.modelo_referencial_generado):
            print("Debe generar ambos modelos para obtener resultados comparativos")
            return
        resultado_referencial, resultado_final = self.obtener_resultados_referencial_vs_final()
        for i in range(self.obtener_num_clases()):
            print("***************")
            print("*** CLASE " + str(i+1) + " ***")
            print("***************")

            acc_mr = resultado_referencial[i]
            acc_mf = resultado_final[i]

            print("Accuracy modelo referencial: " + '{:.1%}'.format(acc_mr))
            print("Accuracy modelo final: " + '{:.1%}'.format(acc_mf))
            print("")

        print("********************")
        print("*** MODELO FINAL ***")
        print("********************")  
        print("Accuracy modelo final: " + '{:.1%}'.format(self.devolver_resultado()["accuracy"]))
        print("")
        
    def generar_predictor_final(self):
        predictor = self.devolver_mejor_modelo()
        llave_fold_final = self.obtener_llaves_clases()[0]
        nuevo_generador_features = GeneradorFeaturesParaPredicciones(carpeta_base=self.carpeta_base, diamond_db=self.diamond_db, carpeta_cpat=self.carpeta_fold_cpat(llave_fold_final))
        predictor.steps.pop(0)
        predictor.steps.insert(0,['features', nuevo_generador_features])
        dump(predictor, self.carpeta_modelo() + "/modelo_final.plk")
        
    def reportar_predicciones(self, archivo_lncRNA, archivo_PCT):
        y_pred_lncRNA = self.realizar_predicciones(archivo_lncRNA)
        probs_lncRNA = self.realizar_predicciones_proba(archivo_lncRNA, features_calculados=True)
        y_pred_PCT = self.realizar_predicciones(archivo_PCT)
        probs_PCT = self.realizar_predicciones_proba(archivo_PCT, features_calculados=True)
        
        y_true = ([1] * len(y_pred_lncRNA)) + ([0] * len(y_pred_PCT))
        y_pred = np.concatenate((y_pred_lncRNA, y_pred_PCT))
        
        plt.figure(1)
        probs = np.concatenate((probs_lncRNA[:,1], probs_PCT[:,1]))
        precision, recall, _ = precision_recall_curve(y_true, probs)
        average_precision = average_precision_score(y_true, probs)
        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
        
        plt.figure(2)
        fpr, tpr, _ = roc_curve(y_true, probs)
        roc_auc = roc_auc_score(y_true, probs)
        plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
        plt.plot([0, 1], [0, 1], linestyle='--')  # random predictions curve
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate or (1 - Specifity)')
        plt.ylabel('True Positive Rate or (Sensitivity)')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        
        return classification_report(y_true, y_pred, target_names=["PCT", "lncRNA"]), confusion_matrix(y_true, y_pred), precision_recall_fscore_support(y_true, y_pred, average='binary')
        
    def reportar_predicciones_keras(self, archivo_lncRNA, archivo_PCT):
        y_pred_lncRNA = self.realizar_predicciones(archivo_lncRNA)
        probs_lncRNA = self.realizar_predicciones_proba_keras(archivo_lncRNA, features_calculados=True)
        y_pred_PCT = self.realizar_predicciones(archivo_PCT)
        probs_PCT = self.realizar_predicciones_proba_keras(archivo_PCT, features_calculados=True)
        
        y_true = ([1] * len(y_pred_lncRNA)) + ([0] * len(y_pred_PCT))
        y_pred = np.concatenate((y_pred_lncRNA, y_pred_PCT))
        
        plt.figure(1)
        probs = np.concatenate((probs_lncRNA[:,1], probs_PCT[:,1]))
        precision, recall, _ = precision_recall_curve(y_true, probs)
        average_precision = average_precision_score(y_true, probs)
        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
        
        plt.figure(2)
        fpr, tpr, _ = roc_curve(y_true, probs)
        roc_auc = roc_auc_score(y_true, probs)
        plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
        plt.plot([0, 1], [0, 1], linestyle='--')  # random predictions curve
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate or (1 - Specifity)')
        plt.ylabel('True Positive Rate or (Sensitivity)')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        
        return classification_report(y_true, y_pred, target_names=["PCT", "lncRNA"]), confusion_matrix(y_true, y_pred), precision_recall_fscore_support(y_true, y_pred, average='binary')
        
    def realizar_predicciones(self, archivo_fasta, features_calculados=False):
        predictor = load(self.carpeta_modelo() + "/modelo_final.plk")
        X_test = util_fasta.leer_fasta_list(archivo_fasta)
        predictor.set_params(features__features_calculados=features_calculados)
        return predictor.predict(X_test)
        
    def realizar_predicciones_proba(self, archivo_fasta, features_calculados=False):
        predictor = load(self.carpeta_modelo() + "/modelo_final.plk")
        X_test = util_fasta.leer_fasta_list(archivo_fasta)
        predictor.set_params(features__features_calculados=features_calculados)
        return predictor.decision_function(X_test)
        
    def realizar_predicciones_proba_keras(self, archivo_fasta, features_calculados=False):
        predictor = load(self.carpeta_modelo() + "/modelo_final.plk")
        X_test = util_fasta.leer_fasta_list(archivo_fasta)
        predictor.set_params(features__features_calculados=features_calculados)
        return predictor.predict_proba(X_test)
        
#wrappers para ejecución en paralelo
def wrapper_armar_fold_final(tesis2, tipo):
    tesis2.armar_fold_final(tipo)
        
def wrapper_armar_fold_clase(tesis2, num_clase):
    tesis2.armar_fold_clase(num_clase)
    
def wrapper_generar_cpat_fold(tesis2, num_clase):
    tesis2.generar_cpat_fold(num_clase)
    
def wrapper_limpieza_archivos_CDS(tesis2, num_clase):
    tesis2.limpieza_archivos_CDS(num_clase)
    
def wrapper_ejecutar_cpat_fold(tesis2, num_clase):
    tesis2.ejecutar_cpat_fold(num_clase)
    
def wrapper_generar_features_fold(tesis2, num_clase):
    tesis2.generar_features_fold(num_clase)
    
def wrapper_limpieza_archivos_finales_fasta(tesis2, num_clase):
    tesis2.limpieza_archivos_finales_fasta(num_clase)

def wrapper_crear_modelo_referencial(tesis2, num_clase):
    tesis2.crear_modelo_referencial(num_clase)
                                         
def crear_modelo_keras(optimizer='adam', learn_rate=0.01, momentum=0, init_mode='uniform', activation='relu', activation2='softmax', dropout_rate=0.0, weight_constraint=1, neurons=10, hidden_layers=1, hidden_neurons=10):
    #optimizer = SGD(lr=learn_rate, momentum=momentum)
    model = Sequential()
    model.add(Dense(units=neurons, activation=activation, input_dim=10, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    for _ in range(hidden_layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(units=hidden_neurons, activation=activation2, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid', kernel_initializer=init_mode))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

class GeneradorFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, tesis2=None, cantidad_transcritos=None, num_clases=None):
        if cantidad_transcritos is None:
            return
        self.cantidad_transcritos = cantidad_transcritos
        self.num_clases = num_clases
        self.tesis2 = tesis2

    def fit(self, X, y=None):
        self._llave_fold = self.obtener_llave_fold(X)
        return self

    def transform(self, X):
        return self.obtener_features_pre_calculados(X)

    def obtener_llave_fold(self, X):
        cod_secuencias = ""
        num_transcritos = len(X)
        num_transcritos_por_grupo = self.cantidad_transcritos
        for i in range(num_transcritos//(num_transcritos_por_grupo*2)):
            cod_secuencias += X[i * num_transcritos_por_grupo][0]
        llave = hashlib.sha224(cod_secuencias.encode()).hexdigest()
        return llave

    def obtener_features_pre_calculados(self, X):
        llave = self._llave_fold
        tipo = "train"
        if os.path.isfile(self.tesis2.archivo_fold_clase(llave, "test", "lncRNA")):
            secuencias = util_fasta.leer_fasta(self.tesis2.archivo_fold_clase(llave, "test", "lncRNA"), 1)
            if list(secuencias.keys())[0] == X[0][0]:
                tipo = "test"
        features = list(load(self.tesis2.archivo_features_clase(llave, tipo, "lncRNA")).values())
        features += list(load(self.tesis2.archivo_features_clase(llave, tipo, "PCT")).values())
        return [list(x.values()) for x in features]
    
class GeneradorFeaturesKeras(BaseEstimator, TransformerMixin):
    def __init__(self, tesis2=None, cantidad_transcritos=None, num_clases=None):
        if cantidad_transcritos is None:
            return
        self.cantidad_transcritos = cantidad_transcritos
        self.num_clases = num_clases
        self.tesis2 = tesis2

    def fit(self, X, y=None):
        self._llave_fold = self.obtener_llave_fold(X)
        return self

    def transform(self, X):
        return self.obtener_features_pre_calculados(X)

    def obtener_llave_fold(self, X):
        cod_secuencias = ""
        num_transcritos = len(X)
        num_transcritos_por_grupo = self.cantidad_transcritos
        for i in range(num_transcritos//(num_transcritos_por_grupo*2)):
            cod_secuencias += X[i][0]
        llave = hashlib.sha224(cod_secuencias.encode()).hexdigest()
        return llave

    def obtener_features_pre_calculados(self, X):
        llave = self._llave_fold
        tipo = "train"
        if os.path.isfile(self.tesis2.archivo_fold_clase(llave, "test", "lncRNA")):
            secuencias = util_fasta.leer_fasta(self.tesis2.archivo_fold_clase(llave, "test", "lncRNA"), 1)
            if list(secuencias.keys())[0] == X[0][0]:
                tipo = "test"
        features = {**load(self.tesis2.archivo_features_clase(llave, tipo, "lncRNA")), **load(self.tesis2.archivo_features_clase(llave, tipo, "PCT"))}
        return [list(features[x[0]].values()) for x in X]

class GeneradorFeaturesParaPredicciones(BaseEstimator, TransformerMixin):
    def __init__(self, carpeta_base=None, diamond_db=None, carpeta_cpat=None, features_calculados=False):
        if carpeta_base is None:
            return
        self.carpeta_base = carpeta_base
        self.diamond_db = diamond_db
        self.carpeta_cpat = carpeta_cpat
        self.features_calculados = features_calculados

    def fit(self, X, y=None):
        raise Exception('Este modelo no admite fit')
        return self

    def transform(self, X):
        carpeta_transform = self.carpeta_base + "/transform"
        if not os.path.isdir(carpeta_transform):
            os.mkdir(carpeta_transform)
        archivo_fasta = carpeta_transform + "/secuencias.fa"
        if (not self.features_calculados):
            self.generar_archivos_fasta(archivo_fasta, X)
            self.ejecutar_diamond_cpat(archivo_fasta)
            self.generar_features(archivo_fasta)
        return self.obtener_features_pre_calculados(archivo_fasta)
                          
    def generar_archivos_fasta(self, archivo_fasta, X):
        util_fasta.generar_fasta(X, archivo_fasta)
    
    def ejecutar_diamond_cpat(self, archivo_fasta):
        diamond_db = self.diamond_db
        carpeta_cpat = self.carpeta_cpat
        util_caracteristicas.ejecutar_diamond(archivo_fasta, diamond_db, archivo_fasta.replace(".fa", ".dmnd"))
        util_caracteristicas.ejecutar_cpat(archivo_fasta, carpeta_cpat, archivo_fasta.replace(".fa", ".cpat"))
        os.remove(archivo_fasta.replace(".fa", ".cpat") + ".dat")
        os.remove(archivo_fasta.replace(".fa", ".cpat") + ".r")
        
    def generar_features(self, archivo_fasta):
        util_caracteristicas.generar_features_base(archivo_fasta, archivo_fasta.replace(".fa", ".cpat"), archivo_fasta.replace(".fa", ".dmnd"), archivo_fasta.replace(".fa", ".ft"))

    def obtener_features_pre_calculados(self, archivo_fasta):
        features = list(load(archivo_fasta.replace(".fa", ".ft")).values())
        return [list(x.values()) for x in features]

Overwriting ./libs/util_modelo_final.py
